In [ ]:
!pip install psycopg2-binary sqlalchemy

In [ ]:
!pip install gradio

In [ ]:
!pip install openai==0.28

In [ ]:
!pip install requests

In [ ]:
!pip install huggingface_hub

In [ ]:
!pip install deep-translator

In [ ]:
from deep_translator import GoogleTranslator

In [ ]:
from sqlalchemy import create_engine, Column, Integer, String, DateTime, Float
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker
from datetime import datetime
import os
import openai
import requests
from transformers import AutoModelForSequenceClassification, AutoTokenizer
import torch
from transformers import AutoConfig
import gradio as gr

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
#модель для детекции
model = AutoModelForSequenceClassification.from_pretrained("rabiaqayyum/autotrain-mental-health-analysis-752423172", use_auth_token=True)
tokenizer = AutoTokenizer.from_pretrained("rabiaqayyum/autotrain-mental-health-analysis-752423172", use_auth_token=True)
config = AutoConfig.from_pretrained("rabiaqayyum/autotrain-mental-health-analysis-752423172", use_auth_token=True)

In [ ]:
class_labels = [config.id2label[i] for i in range(len(config.id2label))]
depression_threshold = 0.9
message_history = []
anomaly_detected = False
message_chat_history=[]
value_crystal = 0
visible_flg = False
mascots = ["Kiko", 'Lori']
openai.api_key = "xxxx"

# Функции

In [ ]:
# модель детекции
def classify_message(user_message, message_history):
    #session = SessionLocal()
   # try:
    detected_lang = user_message
    translated_message = user_message
    inputs = tokenizer([user_message], return_tensors="pt", padding=True, truncation=True)
    outputs = model(**inputs)
    probabilities = outputs.logits.softmax(dim=-1)
    predicted_class_indices = probabilities.argmax(dim=-1)
    depression_probability = probabilities[0][class_labels.index("depression")].item()

    if predicted_class_indices.item() == class_labels.index("depression") and depression_probability < depression_threshold:
        predicted_class = "mentalhealth"
    else:
        predicted_class = class_labels[predicted_class_indices.item()]

    #сохранение сообщения в бд
    #сuser_message_record = UserMessages(user_id=1, message=user_message, classification=predicted_class)
    #с session.add(user_message_record)
    #сsession.commit()

    message_history.append(predicted_class)
    mental_state_count = sum(1 for msg in message_history if msg != "mentalhealth")

    if mental_state_count > 10:
        if mental_state_count > 10:
          warning_message = """We noticed you've been sharing some thoughts that might indicate you're going through a tough time. We recommend seeking help from a professional.
          Sources utiles:
          https://findahelpline.com/
          https://www.mind.org.uk/
          https://www.headspace.com/"""
        return warning_message, True

    return None, False
    #finally:
       # session.close()

Ответ модели

In [ ]:
def respond(user_message, message_history, mascot):
    prompt = f"You are a {mascot}, you are a wise cat helping people improve their mental health. Here is a message from the user: {user_message}. \
    Answer according to your character. Answer in the same language that the user writes in. \
    The first message you send will always be the same: Hello! My name is {mascot}, \
    I am your friend and assistant. We have credited you 1 crystal for admission. \
    Keep it up ! I will be glad to help! How are you feeling today?"
    history_messages = " ".join(message_history)
    prompt_with_history = f"{prompt}\n{history_messages}\n User: {user_message}\n"
    response = openai.ChatCompletion.create(
        model="gpt-4o",
        messages=[{"role": "user", "content": prompt_with_history}]
    )

    #обновление кристаллов
    #session = SessionLocal()
   # try:
    #    #crystal_record = session.query(Crystals).filter(Crystals.user_id == 1).first()
    #    if crystal_record:
     #       crystal_record.crystal_count += 1
      #  else:
       #     crystal_record = Crystals(user_id=1, crystal_count=1)
        #    session.add(crystal_record)
       # session.commit()
    #finally:
     #   session.close()

    return response['choices'][0]['message']['content']

Интерфейс

In [ ]:
def chatbot_interface(user_message, mascot_choice):
    global anomaly_detected, message_history, value_crystal, visible_flg

    if not user_message.strip():
        return "Enter your message here", None

    if not anomaly_detected:
        warning_message, anomaly_detected = classify_message(user_message, message_history)
        if anomaly_detected:
            message_chat_history.extend([f"I: {user_message}", f"{mascot_choice}: {warning_message}"])
            return  None, '\n'.join(message_chat_history)
    if len(message_chat_history) == 0:
      value_crystal=int(value_crystal.value)+1
      visible_flg = True
    response_message = respond(user_message, message_history, mascot_choice)
    message_history.extend([f"User: {user_message}", f"{mascot_choice}: {response_message}"])
    if "Lori:" in response_message or "Kiko:" in response_message:
      response_message = response_message.replace("Lori:", "")
      response_message =  response_message.replace("Kiko:", "")
    message_chat_history.extend([f"I: {user_message}", f"{mascot_choice}: {response_message}"])
    return  None, '\n'.join(message_chat_history), value_crystal, visible_flg

Визуализируем бота

In [ ]:
with gr.Blocks() as iface:
    with gr.Row():
        gr.Image("/content/cristall.png", show_label=False, width=12, height=80, show_download_button = False, visible = True)
        value_crystal = gr.Textbox(label="💎(your crystal count)", value = 0, autoscroll=False)
    gr.Markdown("# Chatbot for mental health")
    gr.Image("/content/logo.jfif", show_label=False, width=2, height=150, show_download_button = False)
    gr.Markdown("# Select a mascot, enter your message to get support and say hi for me :)")

    with gr.Row():
        with gr.Column(scale=3):
            mascot_choice = gr.Dropdown(choices=mascots, label="Choose a mascot", elem_id="mascot-dropdown")
            user_message = gr.Textbox(label = 'Enter message ♡', lines=2, placeholder="Enter your message here...", elem_id="chat-message")
            submit_btn = gr.Button("Send")
        with gr.Column(scale=1):
            mascot_image = gr.Image(label="Mascot", elem_id="mascot-image", show_download_button = False, show_label = False)

    chat_history = gr.Textbox(label="Chat")

    def update_mascot(mascot):
        return f"/content/{mascot}.gif"


    mascot_choice.change(
        fn=update_mascot,
        inputs=[mascot_choice],
        outputs=[mascot_image]
    )

    submit_btn.click(
        fn=chatbot_interface,
        inputs=[user_message, mascot_choice],
        outputs=[user_message, chat_history, value_crystal]
    )
iface.launch(debug = True)
#iface.launch(auth=('user', 'admin'), auth_message="Enter your username and password, please", share=True, debug = True)

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://77b3d25a2ab2f79ede.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


# новая функция детекции

In [ ]:
from deep_translator import GoogleTranslator

def classify_message(user_message, message_history):
    #session = SessionLocal()
    detected_language = GoogleTranslator(source='auto', target='en').detect(user_message)
    try:
        # Перевод сообщения на английский язык
        translated_message = GoogleTranslator(source='auto', target='en').translate(user_message)

        inputs = tokenizer([translated_message], return_tensors="pt", padding=True, truncation=True)
        outputs = model(**inputs)
        probabilities = outputs.logits.softmax(dim=-1)
        predicted_class_indices = probabilities.argmax(dim=-1)
        depression_probability = probabilities[0][class_labels.index("depression")].item()

        if predicted_class_indices.item() == class_labels.index("depression") and depression_probability < depression_threshold:
            predicted_class = "mentalhealth"
        else:
            predicted_class = class_labels[predicted_class_indices.item()]

        #сохранение сообщения в бд
        #user_message_record = UserMessages(user_id=1, message=user_message, classification=predicted_class)
        #session.add(user_message_record)
        #session.commit()

        message_history.append(predicted_class)
        mental_state_count = sum(1 for msg in message_history if msg != "mentalhealth")

        if mental_state_count > 10:
            if mental_state_count > 10:
              warning_message = "We noticed you've been sharing some thoughts that might indicate you're going through a tough time. We recommend seeking help from a professional."
              translated_warning = GoogleTranslator(source='en', target=detected_language).translate(warning_message)

              sourc = """Sources utiles:
              https://findahelpline.com/
              https://www.mind.org.uk/
              https://www.headspace.com/"""
              mes_for_user = translated_warning + "\n\n" + sourc
            return mes_for_user, True

        return None, False
    finally:
        #session.close()

In [ ]:
def respond(user_message, message_history, mascot):
    if not message_history:
        prompt = f"You are a {mascot}, you are a wise cat helping people improve their mental health. The first message you send will always be the same: Hello! My name is {mascot}, I am your friend and assistant. We have credited you 1 crystal for admission. Keep it up! I will be glad to help! How are you feeling today?"
    else:
        prompt = f"You are a {mascot}, you are a wise cat helping people improve their mental health. Here is a message from the user: {user_message}. Answer according to your character. Answer in the same language that the user writes in."

    history_messages = " ".join(message_history)
    prompt_with_history = f"{prompt}\n{history_messages}\n User: {user_message}\n"
    response = openai.ChatCompletion.create(
        model="gpt-4o",
        messages=[{"role": "user", "content": prompt_with_history}]
    )

    #обновление кристаллов
    #session = SessionLocal()
    try:
        #crystal_record = session.query(Crystals).filter(Crystals.user_id == 1).first()
        if crystal_record:
            crystal_record.crystal_count += 1
        else:
            crystal_record = Crystals(user_id=1, crystal_count=1)
            session.add(crystal_record)
        session.commit()
    finally:
        session.close()

    return response['choices'][0]['message']['content']